In [3]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [11]:
import asyncio 
from google import genai
from google.genai import types
import nest_asyncio
nest_asyncio.apply()

client = genai.Client()
model = "gemini-live-2.5-flash-preview"


def open_shop():
    return "opened"

def close_shop():
    return "closed"




shop_open = types.FunctionDeclaration(name="open_shop",behavior="NON_BLOCKING")
shop_close = types.FunctionDeclaration(name="close_shop")


tools = [types.Tool(function_declarations=[shop_open, shop_close])]


#tools = [{"function_declarations":[shop_open,shop_close]}]
config = types.LiveConnectConfig(
     response_modalities = ["TEXT"],
     tools=tools
)


async def main():
    async with client.aio.live.connect(model=model, config = config) as session:
        prompt = "open the shop and tell me what happened"
        await session.send_client_content(turns ={"parts":[{"text":prompt}]})

        async for chunk in session.receive():
            #print(chunk)
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text,end="")
            if chunk.tool_call:
                function_responses = []
                for fc in chunk.tool_call.function_calls:
                    #print(fc)
                    result = open_shop() if fc.name == "open_shop" else close_shop()
                    function_response = types.FunctionResponse(
                        id = fc.id,
                        name = fc.name,
                        response = {"result":result,"scheduling":"WHEN_IDLE"}
                    )
                    function_responses.append(function_response)
                await session.send_tool_response(function_responses=function_responses)
                #await session.send_tool_response(function_responses)

await main()

id='function-call-11038371609359629807' args={} name='open_shop'
The shop is now open.

In [5]:
# Code execution

import asyncio
from google import genai
from google.genai import types
import nest_asyncio
nest_asyncio.apply()


client = genai.Client()
model = "gemini-live-2.5-flash-preview"


tools = [{'code_execution':{}}]
config = {"response_modalities":["Text"],"tools":tools}

async def main():
    async with client.aio.live.connect(model=model,config=config) as session:
        prompt = "Compute the code for face detection using open cv"
        await session.send_client_content(turns={"parts":[{"text":prompt}]})

        async for chunk in session.receive():
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text)
                model_turn = chunk.server_content.model_turn
                if model_turn :
                    for part in model_turn.parts:
                        if part.executable_code is not None:
                            print(part.executable_code.code)
                        if part.code_execution_result is not None:
                            print(part.code_execution_result.output)



await main()

To
 detect faces using OpenCV, you typically follow these steps:

1.  **
Load the pre-trained Haar Cascade classifier:** OpenCV provides pre-trained Haar Cascades for various object detections, including frontal faces.
2.  **Load the image:** Read the image on which you want to perform face detection.
3.  
**Convert to grayscale:** Haar Cascades work best with grayscale images, as color information is not crucial for feature detection in this method.
4.  **Perform face detection:** Use the loaded classifier to detect faces in the grayscale image. This will return
 a list of rectangles, where each rectangle represents a detected face.
5.  **Draw bounding boxes:** Iterate through the detected faces and draw a rectangle (bounding box) around each face on the original image.
6.  **Display
 the image:** Show the image with the detected faces.

Here's the Python code to achieve this:




import cv2
import matplotlib.pyplot as plt

# Step 1: Load the pre-trained Haar Cascade classifier for face detection
# OpenCV provides these XML files. You might need to adjust the path if it's not
# in the same directory as your script or if you have a different OpenCV installation.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Step 2: Load the image
# For demonstration, let's create a dummy image. In a real scenario, you would
# load an image from a file, e.g., image = cv2.imread('your_image.jpg')
# If you are running this locally, make sure to replace 'path/to/your/image.jpg'
# with the actual path to an image containing faces.
# If you don't have an image, let's create a placeholder or assume one will be provided.

# IMPORTANT: If you want to test this code, you need to provide an actual image file.
# For now, let's assume 'test_image.jpg' exists in the same directory.
# If you don't have an image, please upload one.
try:
  

import cv2
import matplotlib.pyplot as plt
import os

# Step 1: Load the pre-trained Haar Cascade classifier for face detection
# Assuming 'haarcascade_frontalface_default.xml' is in the same directory as the script.
# If you haven't downloaded it, please follow the instructions above to get it.
cascade_path = 'haarcascade_frontalface_default.xml'

if not os.path.exists(cascade_path):
    print(f"Error: Haar Cascade file '{cascade_path}' not found.")
    print("Please download 'haarcascade_frontalface_default.xml' from:")
    print("https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml")
    print("And place it in the same directory as this script.")
    # Exit or handle the error appropriately if the cascade file is essential.
    # For now, let's just create a dummy cascade to avoid crashing, but it won't work.
    face_cascade = None
else:
    face_cascade = cv2.CascadeClassifier(cascade_path)


# Step 2: Load the image
image_path = 'test

import cv2
import matplotlib.pyplot as plt
import os

# Ensure these files are in the same directory as this script.
cascade_filename = 'haarcascade_frontalface_default.xml'
image_filename = 'test_image.jpg'

# 1. Load the Haar Cascade classifier
if not os.path.exists(cascade_filename):
    print(f"ERROR: Haar Cascade file '{cascade_filename}' not found.")
    print("Please download it from https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml and place it in the same directory.")
    exit() # Stop execution as the classifier is essential
else:
    face_cascade = cv2.CascadeClassifier(cascade_filename)

# 2. Load the image
if not os.path.exists(image_filename):
    print(f"ERROR: Image file '{image_filename}' not found.")
    print("Please place an image file (e.g., a photo with faces) named 'test_image.jpg' in the same directory.")
    exit() # Stop execution as the image is essential
else:
    image = cv2.imread(image_filename)

# Make a c

import cv2
import matplotlib.pyplot as plt
import os

# Ensure these files are in the same directory as this script.
cascade_filename = 'haarcascade_frontalface_default.xml'
image_filename = 'test_image.jpg'

# 1. Load the Haar Cascade classifier
if not os.path.exists(cascade_filename):
    print(f"ERROR: Haar Cascade file '{cascade_filename}' not found.")
    print("Please download it from https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml and place it in the same directory.")
    exit() # Stop execution as the classifier is essential
else:
    face_cascade = cv2.CascadeClassifier(cascade_filename)

# 2. Load the image
if not os.path.exists(image_filename):
    print(f"ERROR: Image file '{image_filename}' not found.")
    print("Please place an image file (e.g., a photo with faces) named 'test_image.jpg' in the same directory.")
    exit() # Stop execution as the image is essential
else:
    image = cv2.imread(image_filename)

# Make a c

In [6]:
# multiple turn , grounding , code_execution and own function

import asyncio
from google import genai
from google.genai import types
import nest_asyncio
nest_asyncio.apply()
import json 


client = genai.Client()
model = "gemini-live-2.5-flash-preview"

def last_report(output:int):
    return "pass" if output%2==0 else "fail"

last_report_decl = types.FunctionDeclaration(name="last_report")
tools = [{"google_search":{}},
         {'code_execution':{}},
         types.Tool(function_declarations=[last_report_decl])]
config = types.LiveConnectConfig(
    response_modalities=["TEXT"],
    tools=tools)


async def main():
    async with client.aio.live.connect(model=model,config=config) as session:
        prompt = """
        Generate Python code that calculates the total number of Saturdays in 2025
        and prints only a JSON object like {"saturday_count": 52}
        """

        await session.send_client_content(turns={"parts":[{"text":prompt}]})
        output = None
        async for chunk in session.receive():
            if chunk.server_content:
                if chunk.text is not None:
                    print(chunk.text)
            model_turn = chunk.server_content.model_turn
            if model_turn :
                for part in model_turn.parts:
                    if part.executable_code is not None:
                        print(part.executable_code.code)
                    if part.code_execution_result is not None:
                        print(part.code_execution_result.output)
                        data = json.loads(part.code_execution_result.output)
                        output = data["saturday_count"]
            if output is not None:
                break
        prompt1 = f"use the count value of the executed code as input and return the last report"
        await session.send_client_content(turns={"parts":[{"text":prompt1}]},turn_complete=True)

        async for chunk in session.receive():
            if chunk.tool_call:
                for fc in chunk.tool_call.function_calls:
                    result = last_report(output) if fc.name=="last_report" else None

                    function_response = types.FunctionResponse(
                        id = fc.id,
                        name = fc.name,
                        response = {"result":result},
                        scheduling = "WHEN_IDLE"
                    )
                await session.send_tool_response(function_responses=[function_response])
                    
            if chunk.server_content:
                if chunk.text:
                    print(" Model Output:", chunk.text)

            # Token usage
            if chunk.usage_metadata:
                usage = chunk.usage_metadata
                print(f"\nUsed {usage.total_token_count} tokens total.")
                for detail in usage.response_tokens_details:
                    match detail:
                        case types.ModalityTokenCount(modality=modality, token_count=count):
                            print(f"{modality}: {count}")
                
                
                        



await main()

import calendar
import json

saturday_count = 0
for month in range(1, 13):
    cal = calendar.monthcalendar(2025, month)
    for week in cal:
        if week[calendar.SATURDAY] != 0:
            saturday_count += 1

result = {"saturday_count": saturday_count}
print(json.dumps(result))
{"saturday_count": 52}

 Model Output: ```json
 Model Output: 
{"saturday_count": 52}

 Model Output: ```

Used 1410 tokens total.
MediaModality.TEXT: 108


In [1]:
# coroutines

import asyncio
import time
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():

    # The asyncio.create_task() function to run coroutines concurrently as asyncio Tasks.
    task1 = asyncio.create_task(say_after(2,'hello'))
    task2 = asyncio.create_task(say_after(3,'world'))
    print(time.strftime('%X'))
    await task1
    await task2
    print(time.strftime('%X'))


await main()

22:18:48
hello
world
22:18:51


In [2]:
# Awaitables
import asyncio
import time


async def nested():
    return 46

 
# coroutines:
async def main():
    #nested()
    print( await nested())

await main()


# Tasks
async def main():
    task = asyncio.create_task(nested())
    result =  await task
    print(result)

await main()



# Futures

async def main():
    loop = asyncio.get_running_loop()
    future =  loop.create_future()

    loop.call_later(60, future.set_result,"completed")
    print(time.strftime('%X'))
    print("waiting to call ")
    result = await future
    print(result)
    print(time.strftime('%X'))

await main()

    

46
46
22:19:14
waiting to call 
completed
22:20:14


In [4]:
import asyncio, os
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"
handle = open("handle.txt").read().strip() if os.path.exists("handle.txt") else None

async def main():
    cfg = types.LiveConnectConfig(response_modalities=["TEXT"], session_resumption={"handle": handle})
    async with client.aio.live.connect(model=MODEL, config=cfg) as s:
        while (msg := input("You: ")) != "q":
            await s.send_client_content(turns={"role": "user", "parts": [{"text": msg}]})
            async for r in s.receive():
                if r.text: print( r.text)
                if r.session_resumption_update:
                    new_handle = r.session_resumption_update.new_handle
                    if new_handle:
                        with open("handle.txt", "w") as f:
                            f.write(new_handle)
                    break
        
            

await main()


You:  hi how are you
You:  tell about AI in 5 points


I'm doing
 well, thank you for asking! How are you today?


You:  waiting for response
You:  .
You:  .


Okay
, here are 5 key points about AI:

1.  **Simulation
 of Human Intelligence:** At its core, AI aims to create machines that can simulate or mimic human cognitive functions like learning, problem-solving, decision-making, perception, and language understanding. It's about getting computers to "think" in
 some capacity.

2.  **Powered by Data and Algorithms:** AI doesn't work by magic. It heavily relies on vast amounts of data (which it uses to identify patterns and learn from) and sophisticated algorithms (the sets of rules and
 instructions that guide its operations and decision-making processes). More data often means better AI.

3.  **Broad Applications Across Industries:** AI is not a niche technology; it's transforming nearly every sector. Examples include healthcare (diagnosis
, drug discovery), finance (fraud detection, algorithmic trading), retail (recommendations, customer service), manufacturing (automation, quality control), and entertainment (content recommendation, game

You:  q


In [5]:
import asyncio, os
from google import genai
from google.genai import types

client = genai.Client()
MODEL = "gemini-live-2.5-flash-preview"
handle = open("handle.txt").read().strip() if os.path.exists("handle.txt") else None

"""
async def ensures Gemini Live can await the result without blocking the session.

This prevents repeated input prompts because the system waits for the function to finish asynchronously.
"""

async def train_available_timing():
    
    print("morning 4am to 5am")
    await asyncio.sleep(0)
    return "morning 4am to 5am"

function_map = {
    "train_available_timing":train_available_timing,
}

async def run_main():
    cfg = types.LiveConnectConfig(response_modalities=["TEXT"], session_resumption={"handle": handle} if handle else None,
                                 tools = [{"function_declarations":[{"name":"train_available_timing","description":"Provides available train timing for the day.","behavior":"NON_BLOCKING"}]}])
    async with client.aio.live.connect(model=MODEL, config=cfg) as s:
        while True:
            msg = input("You: ")
            if msg.lower() == 'q':
                print("session ended by you")
                break
                
            await s.send_client_content(turns={"role": "user", "parts": [{"text": msg}]})
            async for r in s.receive():
                
                print(r)
                if getattr(r, "tool_call", None):
                    for fn in r.tool_call.function_calls:
                        fn_name = fn.name
                        print(f"Gemini requested: {fn_name}")
                        if fn_name in function_map:
                            result =  await function_map[fn_name]()
                            await s.send_client_content(turns=[{
                                "role": "function",
                                "function_call_id": fn.id,
                                "parts": [
                                    {"text": f"Got it! Here’s today’s train timing: {result}."}
                                ],
                            }])

                elif  getattr(r,"text",None):
                    print( r.text)
                if getattr(r,"session_resumption_update",None):
                    new_handle = r.session_resumption_update.new_handle
                    if new_handle:
                        with open("handle.txt", "w") as f:
                            f.write(new_handle)
                        print("session handle has saved")
        
            

async def main():
    while True:
        try:
            await run_main()
            break
        except Exception as e:
            print(f"Connection dropped ({e}). Reconnecting in 5s...")
            if os.path.exists("handle.txt"):
                os.remove("handle.txt")
            await asyncio.sleep(5)

await main()


You:  hi


setup_complete=None server_content=None tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=LiveServerSessionResumptionUpdate()
setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        text='Hi'
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
Hi
setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        text=""" there! How can I help you today? 
"""
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
 there! How can I help you today? 

setup_complete=None server_content=LiveServerContent(
  generation_complete=True
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
setup_complete=None server_content=LiveServerContent(
  turn_com

You:  train timing


setup_complete=None server_content=None tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=LiveServerSessionResumptionUpdate(
  new_handle='CihicDV0dXc2ZXNrdjFzOWFjNXg1dWgxdHpqeTBhaGE1cGJ3N3FudGZu',
  resumable=True
)
session handle has saved
setup_complete=None server_content=None tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=LiveServerSessionResumptionUpdate()


setup_complete=None server_content=LiveServerContent(
  model_turn=Content(
    parts=[
      Part(
        executable_code=ExecutableCode(
          code='print(default_api.train_available_timing())',
          language=<Language.PYTHON: 'PYTHON'>
        )
      ),
    ]
  )
) tool_call=None tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
setup_complete=None server_content=None tool_call=LiveServerToolCall(
  function_calls=[
    FunctionCall(
      args={},
      id='function-call-835613947163558599',
      name='train_available_timing'
    ),
  ]
) tool_call_cancellation=None usage_metadata=None go_away=None session_resumption_update=None
Gemini requested: train_available_timing
morning 4am to 5am
Connection dropped (Could not convert input (type "<class 'list'>") to `types.LiveClientContent`). Reconnecting in 5s...


You:  q


session ended by you
